In [7]:
import os
import pandas as pd
import numpy as np
from sys import platform

In [8]:
# train_zip = 'train.zip'
# test_zip  = 'test.zip'
# train_file = 'train.csv'
# test_file = 'test.csv'

In [9]:
# def is_colab():
#     return 'google.colab' in str(get_ipython())

# def check_file(filename):
#     if not os.path.exists(filename):
#         if is_colab():
#             from google.colab import files
#             print(f"Please upload {filename} file")
#             uploaded = files.upload()
#         else:
#             print(f"{filename} not exist, this notebook require {filename} to run")
#     else:
#         print(f"{filename} ready")

# def unzip(src, dest):
#     if not os.path.exists(dest):
#         if platform == "linux" or platform == "linux2":
#             !unzip {src} 
#         elif platform == "win32":
#             !tar -xf {src}
#         else:
#             print(f"Please unzip {src} manually")

# check_file(train_zip)
# check_file(test_zip)

# # System check
# # Unzip data files if not already exist
# unzip(train_zip, train_file)
# unzip(test_zip, test_file)

In [10]:
train_file = "/content/drive/MyDrive/Colab Notebooks/train.csv"
test_file = "/content/drive/MyDrive/Colab Notebooks/test.csv"

In [11]:
train_df, test_df = pd.read_csv(train_file), pd.read_csv(test_file)
train_df.head()

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,NaN
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,NaN
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,NaN
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,NaN
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,NaN


In [12]:
test_df.head()

,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,"02 11, 2017",u04140621,ok,Five Stars,1486771200,Pop,$31.20,p65721979,98944233,NaN
1,"01 29, 2018",u74766187,The 1955 and 1958 sets alone are worth the pur...,Buy It For the 1955 Set With Monk!,1517184000,Jazz,$34.04,p87809670,33903910,NaN
2,"08 29, 2005",u31689638,Mary J Blige is someone most people know of. S...,Mary's excellent debut! Classic!,1125273600,Pop,$1.72,p52702240,35386470,NaN
3,"11 2, 2015",u35613516,Very good satch album,Four Stars,1446422400,Pop,$10.99,p01314374,12404766,NaN
4,"01 3, 2012",u93320378,Well there are hundreds of Christmas anthologi...,CD lives up to its name,1325548800,Alternative Rock,$7.98,p84396269,68253688,NaN


In [13]:
print(f'training set shape: {train_df.shape}')
print(f'testing set shape: {test_df.shape}')

training set shape: (150000, 11)
testing set shape: (20000, 10)


In [14]:
print(f'Number of reviewText nan in training set: {train_df.reviewText.isna().sum()}')
print(f'Number of reviewText nan in testing set: {test_df.reviewText.isna().sum()}')

Number of reviewText nan in training set: 28
Number of reviewText nan in testing set: 3


In [15]:
train_df.dropna(axis=0, subset=['reviewText'], inplace=True)
test_df.dropna(axis=0, subset=['reviewText'], inplace=True)

In [16]:
train_df[train_df.reviewText.isna()==1]

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image


In [17]:
# drop rows with NaN review te

# Preprocess reviewText using NLP

In [18]:
!pip install spacytextblob

     |████████████████████████████████| 10.4MB 7.2MB/s 
     |████████████████████████████████| 1.1MB 50.9MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [20]:
!python -m spacy validate
!python -m spacy download en_core_web_sm

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.5) ======================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                       
package   en-core-web-sm   en_core_web_sm   2.2.5   --> 2.3.1


============================== Install updates ==============================
Use the following commands to update the model packages:
python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.1MB 7.4MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp37-none-any.whl size=12047109 sha256=097bccdfca4c6bb493a71e5985c8ae066075968867e9ce81bbbef6c5512febba
  Stored in directory: /tmp/pip-ephem-wheel-cache-vglm5ouf/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled e

In [21]:
import spacy
import html
import re
from spacytextblob.spacytextblob import SpacyTextBlob
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')
sentencizer = nlp.create_pipe("sentencizer")
spacy_text_blob = SpacyTextBlob()
nlp.add_pipe(spacy_text_blob)
nlp.add_pipe(sentencizer)
all_stopwords = nlp.Defaults.stop_words



def cleanhtml(raw_html):
  cleanr = re.compile('/<[^>]+>/')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def clean(review):
  modReview = review
  # remove html tags and attributes
  modReview = cleanhtml(review)
  # replace newlines with spaces
  modReview = modReview.replace("\n", " ").replace("\r", " ")
  # unescape html
  modReview = html.unescape(modReview)
  # remove URLs
  # reference: https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url
  modReview = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", "", modReview)
  # lower case
  modReview = modReview.lower()
  # remove duplicate spaces
  modReview = " ".join(modReview.split())
  return modReview

def preproc_reviews(nlp, reviews):
  preproced_reviews = []
  polarities = []
  subjectivities = []
  # With as many processes as CPUs (use with caution!)
  for doc in tqdm(nlp.pipe(reviews, n_process=-1)):
    preproced_review = ""
    polarities.append(doc._.sentiment.polarity)
    subjectivities.append(doc._.sentiment.subjectivity)

    if len(doc.text) > 0:
      for sent in doc.sents:
        wordsList = []
        for token in sent:
          if len(token.text.strip()) != 0:
            # tagging
            #    * Write "/POS" after each token.
            tag = token.tag_

            #lemmatization
            if token.lemma_[0] == '-' and token.text[0] != '-':
                lemma = token.text
            else:
                lemma = token.lemma_
            
            # remove stop words
            if not lemma in all_stopwords:
              wordsList.append(lemma + "/" + tag)

        #    * Split tokens with spaces.
        sent = " ".join(wordsList)

        #    * Insert period between sentences.
        if sent and sent[-1] != '.':
            sent += '.'

        preproced_review += sent
        #    * Insert space between sentences.
        preproced_review += " "
      preproced_review = clean(preproced_review)
    preproced_reviews.append(preproced_review)
  return preproced_reviews, polarities, subjectivities

In [22]:
preproced_reviews, polarities, subjectivities = pd.Series(preproc_reviews(nlp, train_df['reviewText']))

In [25]:
train_df['preproced_text'], train_df['polarities'], train_df['subjectivities'] = preproced_reviews, polarities, subjectivities

In [ ]:
train_df['preproced_text'].isna().sum()

28

In [26]:
train_df

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image,preproced_text,polarities,subjectivities
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,NaN,contemporary/jj jazz/nn soul/nn performer/jj -...,0.229849,0.426437
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,NaN,good/jj idea/nn '/`` pop/nn '/'' '/`` orchestr...,0.280937,0.460303
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,NaN,great/jj collection/nn carole/nnp king/nnp son...,0.800000,0.750000
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,NaN,album/nn brilliant/jj piece/nn jazz/nnp fusion...,0.680208,0.800000
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,NaN,wait/vbd long/jj time/nn dvd/nn release/vbn ./...,0.129478,0.440930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,5.0,"09 12, 2013",u29136602,There is just something about Wesley's voice t...,Just right.,1378944000,Alternative Rock,$9.49,p26326919,90575108,NaN,wesley/nnp voice/nn swoon/nn little/jj bit/nn ...,0.195663,0.553061
149996,5.0,"05 15, 2016",u51715193,How many recordings of Verdis Requiem do you o...,Being there in vibrant immediacy of performan...,1463270400,Classical,$14.83,p69247882,74425587,NaN,recording/nns verdis/nnp requiem/nnp ?/. -/hyp...,0.263781,0.590520
149997,5.0,"06 13, 2004",u28696060,....their best overall album. After the revis...,Creative rebirth and possibly....,1087084800,Alternative Rock,$18.86,p07028930,20399056,NaN,..../nfp good/jjs overall/jj album/nn ./. revi...,0.171429,0.457143
149998,5.0,"06 27, 2012",u04925906,"Believe it or not, once upon a time, artists l...",Once Upon A Time,1340755200,Pop,$19.99,p38606785,5931588,NaN,"believe/vb ,/, time/nn ,/, artist/nns like/in ...",0.095939,0.482133


In [27]:
train_df.to_csv("/content/drive/MyDrive/Colab Notebooks/preproced_train_df.csv")